In [2]:
import os
import pandas as pd
import numpy as np
from ezchord import Chord
from mingus.core.notes import int_to_note

from melody import Melody
from utils import get_chord_progressions, is_weakly_polyphonic, is_strongly_polyphonic

In [3]:
bpm = 4

cp = get_chord_progressions('..')['A Felicidade']

linear_chord_progression = []

for section in cp['sections']:
    linear_chord_progression += cp['progression'][section]

n_chord_prog_measures = int(len(linear_chord_progression) / bpm)

In [4]:
orig = pd.read_csv('../data/split_melody_data/v1.2/Real Book/A Felicidade -o-.csv', index_col=0)
orig['end_ticks'] = (orig['ticks'] + orig['duration'])
orig.sort_values('ticks', inplace=True)

In [5]:
orig.shape

(126, 13)

In [6]:
impro = pd.read_csv('../data/split_melody_data/v1.2/JazzPage/A Felicidade -1-.csv', index_col=0)
impro['end_ticks'] = (impro['ticks'] + impro['duration'])
impro.sort_values('ticks', inplace=True)

In [7]:
n_chord_prog_measures

48

In [8]:
def multiple_pitches_to_string(pitches):
    return "-".join(str(x) for x in pitches)

In [9]:
def extract_poly_time_step_encoding(impro, orig, linear_chord_progression):
    tpm = 48
    n_chord_prog_beats = len(linear_chord_progression)
    n_ticks = tpm * n_chord_prog_measures

    rows = []

    for i in range(n_ticks):
        offset = i % tpm

        impro_sustains = impro[(impro['ticks'] < i) & (i < impro['end_ticks'])]['pitch'].values
        impro_sustains = multiple_pitches_to_string(impro_sustains)

        impro_attacks = impro[impro['ticks'] == i]['pitch'].values
        impro_attacks = multiple_pitches_to_string(impro_attacks)

        orig_sustains = orig[(orig['ticks'] < i) & (i < orig['end_ticks'])]['pitch'].values
        orig_sustains = multiple_pitches_to_string(orig_sustains)

        orig_attacks = orig[orig['ticks'] == i]['pitch'].values
        orig_attacks = multiple_pitches_to_string(orig_attacks)

        chord_name = linear_chord_progression[(i // (48 // bpm))]   

        rows.append({
            'offset': offset,

            'impro_sustains': impro_sustains,
            'impro_attacks': impro_attacks,

            'orig_sustains': orig_sustains,
            'orig_attacks': orig_attacks,

            'chord_name': chord_name
        })
        
    return pd.DataFrame(rows)

In [10]:
def remove_strong_polyphony(melody):
    mono_melody = melody.copy()
    
    mono_melody.groupby('ticks').apply(lambda : x['pitch'].max())
    
    return mono_melody

In [11]:
def remove_weak_polyphony(melody):
    mono_melody = melody.copy()

    overlap = (mono_melody['end_ticks'] - mono_melody['ticks'].shift(-1)).clip(0, None)
    

    # skip last row as 'shift' messes it up
    mono_melody['duration'].iloc[:-1] -= overlap.iloc[:-1]
    mono_melody['end_ticks'].iloc[:-1] -= overlap.iloc[:-1]
    

    return mono_melody

In [17]:
def extract_mono_time_step_encoding(impro, orig, linear_chord_progression):
    tpm = 48
    n_chord_prog_beats = len(linear_chord_progression)
    n_ticks = tpm * n_chord_prog_measures

    rows = []

    if is_strongly_polyphonic(impro):
        print('Impro SP')
        impro = remove_strong_polyphony(impro)
        
        if is_strongly_polyphonic(impro):
            raise Exception('Error!!! P')

    if is_weakly_polyphonic(impro):
        print('Impro WP')
        impro = remove_weak_polyphony(impro)
               

        if is_weakly_polyphonic(impro):
            raise Exception('Error!!! W')

    if is_strongly_polyphonic(orig):
        print('Original SP')
        
        orig = remove_strong_polyphony(orig)
        
        if is_strongly_polyphonic(impro):
            raise Exception('Error!!! P')
            
    if is_weakly_polyphonic(orig):
        print('Original WP')
        
        orig = remove_weak_polyphony(orig)

        if is_weakly_polyphonic(orig):
            raise Exception('Error!!! W')

    for i in range(n_ticks):
        offset = i % tpm

        impro_pitch = np.nan
        impro_pitches = impro[(impro['ticks'] <= i) & (i < impro['end_ticks'])]

        if len(impro_pitches) > 0:
            impro_pitch = impro_pitches['pitch'].values[0]

            if len(impro_pitches) > 1:
                raise Exception('Error!!! not mono pitch')
                
        impro_attack = 0
        impro_attacks = impro[impro['ticks'] == i]

        if len(impro_attacks) > 0:
            impro_attack = 1

            if len(impro_attacks) > 1:
                raise Exception('Error!!! not mono attack')

        orig_pitch = np.nan
        orig_pitches = orig[(orig['ticks'] <= i) & (i < orig['end_ticks'])]

        if len(orig_pitches) > 0:
            orig_pitch = orig_pitches['pitch'].values[0]

            if len(orig_pitches) > 1:
                print(orig_pitches[['pitch', 'ticks', 'end_ticks']])
                raise Exception('Error!!! not mono pitch')

        orig_attack = 0
        orig_attacks = orig[orig['ticks'] == i]

        if len(orig_attacks) > 0:
            orig_attack = 1

            if len(orig_attacks) > 1:
                raise Exception('Error!!! not mono attack')
  
        chord_name = linear_chord_progression[(i // (48 // bpm))]   
        
        rows.append({
            'offset': offset,

            'impro_pitch': impro_pitch,
            'impro_attack': impro_attack,

            'orig_pitch': orig_pitch,
            'orig_attack': orig_attack,

            'chord_name': chord_name
        })

    return pd.DataFrame(rows)

In [18]:
mono = extract_mono_time_step_encoding(impro, orig, linear_chord_progression)

Impro WP
Original WP


In [19]:
poly = extract_poly_time_step_encoding(impro, orig, linear_chord_progression)

In [20]:
p = set(poly[poly['orig_attacks']!=''].index)

In [21]:
m = set(mono[mono['orig_attack']!=0].index)

In [22]:
pd.concat([mono, poly],axis=1).loc[list(p.difference(m))][['orig_attacks', 'orig_attack']]

,orig_attacks,orig_attack


In [23]:
import os

mono_filepath = '../data/encoded/timestep/mono/JazzPage/'
if not os.path.exists(mono_filepath):
    os.makedirs(mono_filepath)
    
poly_filepath = '../data/encoded/timestep/poly/JazzPage/'
if not os.path.exists(poly_filepath):
    os.makedirs(poly_filepath)

mono.to_csv(mono_filepath + 'A Felicidade -1-.csv')
poly.to_csv(poly_filepath + 'A Felicidade -1-.csv')

In [ ]:
mono[mono['impro_attack'] == True].tail(5)

In [ ]:
impro.tail(5)

In [ ]:
len(linear_chord_progression)

In [ ]:
mono.tail()